In [35]:
import numpy as np

In [36]:
class CNNLayer:
    
    
    def __init__(n, filter_shape, stride, activation):
        #n is the number of filters, 
        #filter_shape is the 3-dimensional shape of each filter, (height, width, depth), 
        #activation is the type of activation to use
        
        #ReLU implementation: w = np.random.randn(n) * sqrt(2.0/n)
        self.f_n = n
        self.filter_shape = filter_shape
        self.f_h = filter_shape[0]
        self.f_w = filter_shape[1]
        self.f_d = filter_shape[2]
        self.stride = stride
        self.activation = activation
    
    def forward_step(self, X, pad):
        # X is the input data with shape (N, h, w, d)
        # convert X,pad to im column 
        #reshape X so that it's easier to see!
        p = pad
        s = self.stride
        
        
        return forward_step2(X, self.f_n, self.f_h, self.f_w, self.f_d, p, s)
        
    
    def backward_step(self, out_delta):
        # remove pad 
        pass
    
    def update(self):
        """
        for w, gw, b, gb in zip(self.weights, reversed(gws), self.biases, reversed(gbs)):
        w -= self.eta * gw
        b -= self.eta * gb 
        
        """
        pass
    
    def print(self):
        pass
    
    def set_filters(self,filters):
        pass
    


#for individual image requires image to be in d*h*w format
def im2col(X, f_h, f_w, f_d, p, s):
    nr = X.shape[0]
    X_padded = np.pad(X, p, mode='constant')
    X_padded = X_padded[p:(nr+p),:, :]
    
    #print("X_padded and reshaped so d*h*w")
    #print(X_padded)
    #print(X_padded.shape)
    
    X_w = X_padded.shape[2]
    X_h = X_padded.shape[1]
    #print("X.shape 0:{}, 1:{}, 2:{}".format(X_padded.shape[0], X_padded.shape[1], X_padded.shape[2]))
    h=0
    
    X_reshaped = np.zeros((f_w * f_h * f_d,1))
    print("X_reshaped", X_reshaped.shape)
    #print(X_h - f_h)
    while (h < X_h - f_h + 1):
        w= 0
        #print("h")
        while (w < X_w - f_w + 1):
            #print("w")
            X_sub = X_padded[:,h:h+f_h, w:w+f_w]
            X_stack = np.reshape(X_sub,(-1,1))
            X_reshaped = np.hstack((X_reshaped, X_stack))
            w += stride
        h += stride
    #remove first row of zeros
    X_reshaped = X_reshaped[:,1:]
    #print("X_reshaped", X_reshaped.shape)
    return(X_reshaped)

In [85]:
#for single image
#image requires image to be in d*h*w format
def forward_step2(X, f_n, f_h, f_w, f_d, p, s):
    X_reshaped = im2col(X, f_h, f_w, f_d, p, s)
    C_out_h = ((X.shape[2] - f_w + 2*p)/s + 1) #assumes square filter, X.shape[2] used in case n*d*h*w is used in future
    
    #make filter and biases (TODO: get rid of this once filters initialized)
    a = np.arange(f_n*f_d*f_h*f_w)
    b = np.arange(f_n) #biases
    
    #reshape filter into 'F' matrix for final matrix multiplication. 
    a = a.reshape((f_n,f_d,f_h,f_w))#filters in dxhxw format to be more easily readable, 
    F = np.reshape(a, (f_n,-1))
    
    #matrix multiplication
    C = np.dot(F,X_reshaped).T + b
    C = C.T

    C = C.reshape((-1,C_out_h,C_out_h))
    return C
    

In [86]:
def rotate_filter(a): #a.shape n*d*f*w --> return flipped filter (both up-down, and left-right)
    a_flipped = [0,0,0]

    for num in range(a.shape[0]):#for each filter
        for chan in range(a.shape[1]): #for each channel
            fil = a[num, chan, :,:]
            fil_ud = np.flipud(fil)
            fil_lr_ud = np.fliplr(fil_ud)

            a_flipped = np.vstack((a_flipped, fil_lr_ud))
            #print(fil_lr_ud)
    #print(a_flipped)
    print(a_flipped.shape)
    a_flipped = a_flipped[1:, :]
    a_flipped = a_flipped.reshape((f_n,f_d,f_h,f_w))
    return a_flipped

### Test forward (Shows how to reshape X into d*h*w for single and pass single image to forward)
#### do not delete:  above code assumes single image is X

In [87]:
#original images shape
e = np.arange(225).reshape((3,5,5,-1))

#transposed image so that (N,d,h,w), channels print as blocks
nImages = e.transpose((0,3,1,2))

f_h = 3
f_w = 3
f_d = 3
f_n = 2 #2 filters of 3x3x3
p = 1
s = 1

X = nImages[0, :,:,:]


C = forward_step2(X, f_n, f_h, f_w, f_d, p, s)
C #shape is f_n * receptive_field * receptive_field

X_reshaped (27, 1)


/Users/mousehouseapp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


array([[[  2016.,   3348.,   4131.,   4914.,   3408.],
        [  4914.,   7695.,   8748.,   9801.,   6624.],
        [  8559.,  12960.,  14013.,  15066.,   9999.],
        [ 12204.,  18225.,  19278.,  20331.,  13374.],
        [  8136.,  12042.,  12663.,  13284.,   8664.]],

       [[  5257.,   8938.,  11179.,  13420.,   9565.],
        [ 13420.,  21547.,  24787.,  28027.,  19504.],
        [ 24355.,  37747.,  40987.,  44227.,  30169.],
        [ 35290.,  53947.,  57187.,  60427.,  40834.],
        [ 25957.,  39502.,  41581.,  43660.,  29401.]]])

In [91]:
def backward_step2(out_delta, X, f_n, f_h, f_w, f_d, p, s):
    padding = int((X.shape[1] - f_w)/ 2)
    out_delta_padded = np.pad(out_delta, padding, mode='constant')
    
    #make filter and biases (TODO: get rid of this once filters initialized)
    a = np.arange(f_n*f_d*f_h*f_w)
    b = np.arange(f_n) #biases

    #reshape filter into 'F' matrix for final matrix multiplication. #example shape = 2 filters of 3*3*3
    a = a.reshape((f_n,f_d,f_h,f_w))#filters in dxhxw format to be more easily readable, 
    a_rotated = rotate_filter(a)
    
    #apply filter to delta to get delta for previous layer
    F = np.reshape(a_rotated, (f_n,-1))
    
    #matrix multiplication
    C = np.dot(F,out_delta_padded).T
    C = C.T

    #C = C.reshape((-1,5,5))
    return C


In [92]:
out_delta = 1
print(backward_step2(out_delta, X, f_n, f_h, f_w, f_d, p, s))

(19, 3)
[[ 8  7  6  5  4  3  2  1  0 17 16 15 14 13 12 11 10  9 26 25 24 23 22 21
  20 19 18]
 [35 34 33 32 31 30 29 28 27 44 43 42 41 40 39 38 37 36 53 52 51 50 49 48
  47 46 45]]


In [90]:
#make filter and biases (TODO: get rid of this once filters initialized)
a = np.arange(f_n*f_d*f_h*f_w)
b = np.arange(f_n) #biases

#reshape filter into 'F' matrix for final matrix multiplication. 
a = a.reshape((f_n,f_d,f_h,f_w))#filters in dxhxw format to be more easily readable, 
a #example shape = 2 filters of 3*3*3

array([[[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])

#### rotate filter

In [76]:
#make filter and biases (TODO: get rid of this once filters initialized)
a = np.arange(f_n*f_d*f_h*f_w)
b = np.arange(f_n) #biases

#reshape filter into 'F' matrix for final matrix multiplication. 
a = a.reshape((f_n,f_d,f_h,f_w))#filters in dxhxw format to be more easily readable, 
a #example shape = 2 filters of 3*3*3

def rotate_filter(a): #a.shape n*d*f*w --> return flipped filter (both up-down, and left-right)
    a_flipped = [0,0,0]

    for num in range(a.shape[0]):#for each filter
        for chan in range(a.shape[1]): #for each channel
            fil = a[num, chan, :,:]
            fil_ud = np.flipud(fil)
            fil_lr_ud = np.fliplr(fil_ud)

            a_flipped = np.vstack((a_flipped, fil_lr_ud))
            #print(fil_lr_ud)
    #print(a_flipped)
    print(a_flipped.shape)
    a_flipped = a_flipped[1:, :]
    a_flipped = a_flipped.reshape((f_n,f_d,f_h,f_w))
    return a_flipped

(19, 3)


array([[[[ 8,  7,  6],
         [ 5,  4,  3],
         [ 2,  1,  0]],

        [[17, 16, 15],
         [14, 13, 12],
         [11, 10,  9]],

        [[26, 25, 24],
         [23, 22, 21],
         [20, 19, 18]]],


       [[[35, 34, 33],
         [32, 31, 30],
         [29, 28, 27]],

        [[44, 43, 42],
         [41, 40, 39],
         [38, 37, 36]],

        [[53, 52, 51],
         [50, 49, 48],
         [47, 46, 45]]]])